# Personal Search

## Multi Query Answering System



## Sistema de resposta Multi Query



## Import libraries

In [1]:
from IPython.display import display, Markdown
from langchain import hub
from search.context import utils

from operator import itemgetter
#from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

## Configuration

In [2]:
# Model from Ollama
model = "dolphin-llama3:8b"

## Retriever Instance

In [3]:
db = utils.retriever_context(vector_store='chroma', path_books='./vector-store/chroma/books')
retriever = db.as_retriever(
            search_type="similarity",
            search_kwargs={'k': 3}
        )

## Prompt

In [4]:
# Multi Query: Different Perspectives
prompt_perspectives = hub.pull(owner_repo_commit="personal-search/multi-query-search")

## Question

In [5]:
question = "O que é o julgamento do trono branco?"

## Search

In [6]:
generate_queries = utils.generate_multi_query(prompt=prompt_perspectives, model=model)

queries_pespectives = generate_queries.invoke({"question":question})

In [7]:
queries_pespectives

['1. Qual é a definição do julgamento do trono branco?',
 '2. O que acontece durante o julgamento do trono branco?',
 '3. Como é a estrutura do julgamento do trono branco?',
 '4. Quais são as implicações do julgamento do trono branco?',
 '5. Por que é importante entender o conceito de julgamento do trono branco?']

In [13]:
# Retrieve
retrieval_chain = generate_queries | retriever.map() | utils.get_unique_union
#retrieval_chain = generate_queries | db.as_retriever() | utils.get_unique_union
#docs = retrieval_chain.invoke({"question":question})
#docs

In [9]:
# RAG
template = """
Contexto:
{context}

Sintetize o contexto respondendo a seguinte pergunta e cite a fonte(source): {question}
"""

prompt = ChatPromptTemplate.from_template(template)

#llm = ChatOpenAI(temperature=0)
llm = Ollama(model=model, temperature=0)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm.bind(stop=['<|eot_id|>'])
    | StrOutputParser()
)

final_rag_answer = final_rag_chain.invoke({"question":question})

/home/cristian/miniconda3/envs/crewai/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


## Anwser

In [10]:
display(Markdown(f"**Busca:** {question}\n\n"))
display(Markdown(final_rag_answer))

**Busca:** O que é o julgamento do trono branco?



O julgamento do trono branco é uma referência bíblica que pode ser encontrada em Apocalipse 20:11-15. É um evento no qual Deus julga os mortos, baseado nas decisões tomadas durante a vida das pessoas. A fonte deste contexto é especificamente o livro "O Peregrino" de John Bunyan, na página 210.

## Debug

In [11]:
sources = retriever.invoke(question)
#sources = db.as_retriever().invoke(question)
for source in sources:
    display(Markdown(f"**Fonte:** {source.metadata}"))
    display(Markdown(f"**Trecho:** {source.page_content}"))

**Fonte:** {'page': 61, 'source': './books/Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf'}

**Trecho:** boa posição perante os homens, se não a temos diante de Deus? De 
que vale a higiene do corpo, se nossa mente e espírito estão sujos? 
De que adianta possuirmos uma fachada de religiosidade se nosso 
coração é carnal? Por que nos orgulharemos de força física, por 
exemplo, se esp iritualmente somos fracos? De que vale a riqueza do

**Fonte:** {'page': 45, 'source': './books/Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf'}

**Trecho:** está em nós mesmos? ” Mas eu gostaria de afiar bem o meu bisturi e 
aprofundá -lo um pouco mais nos pregadores:  passou a época dos 
grandes ser mões tipo “lanche rápido ”, temperados com tiradas 
humorísticas para tentar estimular o fraco apetite espiritual do homem 
de nossos dias? Ou estamos nos esforçando para comunicar os 
“poderes do mundo vindouro ” em todos os cultos?  
Pensemos um pouco em Paulo. Após recebe r uma poderosa unção 
do Espírito Santo, ele saiu pela Ásia menor para travar ali uma 
intensa batalha espiritual, causando agitação  nos mercados, 
sinagogas e palácios. E ia a toda parte, tendo no coração e nos

**Fonte:** {'page': 8, 'source': './books/Sociedade do Cansaço - Byung-Chul Han.pdf'}

**Trecho:** Sumário
1  A violência neuronal
2  Além da sociedade disciplinar
3  O tédio profundo
4  Vita activa
5  Pedagogia do ver
6  O Caso Bartleby
7  Sociedade do cansaço
Textos de capa

In [12]:
sources = db.similarity_search_with_score(question)
for source in sources:
    display(Markdown(f"**Fonte:** {source[0].page_content}"))
    display(Markdown(f"**Trecho:** {source[0].metadata}"))
    display(Markdown(f"**Score:** {source[1]}"))    

**Fonte:** boa posição perante os homens, se não a temos diante de Deus? De 
que vale a higiene do corpo, se nossa mente e espírito estão sujos? 
De que adianta possuirmos uma fachada de religiosidade se nosso 
coração é carnal? Por que nos orgulharemos de força física, por 
exemplo, se esp iritualmente somos fracos? De que vale a riqueza do

**Trecho:** {'page': 61, 'source': './books/Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf'}

**Score:** 395.99456787109375

**Fonte:** está em nós mesmos? ” Mas eu gostaria de afiar bem o meu bisturi e 
aprofundá -lo um pouco mais nos pregadores:  passou a época dos 
grandes ser mões tipo “lanche rápido ”, temperados com tiradas 
humorísticas para tentar estimular o fraco apetite espiritual do homem 
de nossos dias? Ou estamos nos esforçando para comunicar os 
“poderes do mundo vindouro ” em todos os cultos?  
Pensemos um pouco em Paulo. Após recebe r uma poderosa unção 
do Espírito Santo, ele saiu pela Ásia menor para travar ali uma 
intensa batalha espiritual, causando agitação  nos mercados, 
sinagogas e palácios. E ia a toda parte, tendo no coração e nos

**Trecho:** {'page': 45, 'source': './books/Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf'}

...

**Trecho:** {'page': 8, 'source': './books/Sociedade do Cansaço - Byung-Chul Han.pdf'}

**Score:** 400.4464111328125

**Fonte:** Quando, porém, vier o Consolador, que eu vos enviarei da parte do Pai, o Espírito
da verdade, que dele procede, esse dará testemunho de mim; e vós também
testemunhareis, porque estais comigo desde o princípio.
Tenho-vos dito estas coisas para que não vos escandalizeis. Eles vos expulsarão
das sinagogas; mas vem a hora em que todo o que vos matar julgará com isso
tributar culto a Deus. Isto farão porque não conhecem o Pai, nem a mim. Ora, estas
coisas vos tenho dito para que, quando a hora chegar, vos recordeis de que eu vo-
las disse. Não vo-las disse desde o princípio, porque eu estava convosco (Jo 15.24-
16.4).
 
OUTRO CONSOLADOR

**Trecho:** {'page': 210, 'source': './books/Somos Todos Teólogos - RC Sproul.pdf'}

**Score:** 404.67401123046875